## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Aug 13 12:33:20 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,661588,35.4,1454218,77.7,1387145,74.1
Vcells,1231713,9.4,8388608,64.0,1975448,15.1


In [3]:
add_log = function(message){
    cat(message, file = paste0(getwd(), '/logs.txt'),append=TRUE)
}

In [24]:
require("data.table")
require("csv")
if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [25]:
PARAM <- list()
PARAM$semilla_primigenia <- 424243

PARAM$experimento <- "6300_generencia_FE_gen_testB2"
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [26]:
# carpeta de trabajo

experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(paste0("./exp/",experimento_folder), showWarnings=FALSE)
#setwd( paste0("./exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [28]:
# lectura del dataset
dataset <- fread(paste0("./", PARAM$dataset))

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [29]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [30]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [31]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "mpayroll_sobre_edad"

#### 9.7.1.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustando por algunos indices financieros

In [32]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.780952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)

tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[["foto_mes"]] <- vfoto_mes

# tb_indices

In [33]:
## TODO: juntar en una funcion...

drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}
drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


In [34]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios
add_log(campos_monetarios)

[1] "mrentabilidad"           "mrentabilidad_annual"   
 [3] "mcomisiones"             "mactivos_margen"        
 [5] "mpasivos_margen"         "mcuenta_corriente"      
 [7] "mcaja_ahorro"            "mcuentas_saldo"         
 [9] "mtarjeta_visa_consumo"   "mtarjeta_master_consumo"
[11] "mprestamos_personales"   "mpayroll"               
[13] "Master_mpagominimo"      "Visa_mpagominimo"       
[15] "mpayroll_sobre_edad"

In [35]:
# ejecuto el Data Drifting
setorder(dataset, numero_de_cliente, foto_mes)

PARAM$DR$metodo <- "deflacion"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_deflacion()
fin drift_deflacion()


In [36]:
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "mpayroll_sobre_edad"

#### 9.7.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [37]:
if( !require("lubridate")) install.packages("lubridate", repos = "http://cran.us.r-project.org")
require("lubridate")

In [38]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function(patributos)
{
  atributos <- unique(patributos)
  comun <- intersect(atributos, colnames(dataset))

  return( length(atributos) == length(comun))
}

In [39]:
# Esta es la parte que los alumnos deben desplegar todo su ingenio
# Agregar aqui sus PROPIAS VARIABLES manuales

AgregarVariables_IntraMes <- function(dataset) {
  cat( "inicio AgregarVariables_IntraMes()\n")
  add_log( "inicio AgregarVariables_IntraMes()\n")
  gc(verbose= FALSE)
  # INICIO de la seccion donde se deben hacer cambios con variables nuevas

  # el mes 1,2, ..12
  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  # creo un ctr_quarter que tenga en cuenta cuando
  # los clientes hace 3 menos meses que estan
  # ya que seria injusto considerar las transacciones medidas en menor tiempo
  if( atributos_presentes( c("ctrx_quarter") ))
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[
      cliente_antiguedad == 3,
      ctrx_quarter_normalizado := ctrx_quarter * 1.2
    ]

   if(atributos_presentes(c("foto_mes")))
    dataset[,foto_mes_formato_fecha := as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))]

  #dataset$foto_mes_formato_fecha <<- as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))

  if(atributos_presentes(c("cantidad_total_transacciones"))){
   auxiliarmenos1 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha, cantidad_total_transacciones)]
   auxiliarmenos2 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha,cantidad_total_transacciones)]
   # auxiliarmenos1$foto_mes_formato_fecha <- as.Date(auxiliarmenos1$foto_mes_formato_fecha)
   # auxiliarmenos2$foto_mes_formato_fecha <- as.Date(auxiliarmenos2$foto_mes_formato_fecha)
   auxiliarmenos1$foto_mes_formato_fecha <- auxiliarmenos1$foto_mes_formato_fecha  %m-%  months(1)
   auxiliarmenos2$foto_mes_formato_fecha <- auxiliarmenos2$foto_mes_formato_fecha %m-% months(2)
   auxiliarmenos1$codigo <- paste(auxiliarmenos1$numero_de_cliente,auxiliarmenos1$foto_mes_formato_fecha,sep='-')
   auxiliarmenos2$codigo <- paste(auxiliarmenos2$numero_de_cliente,auxiliarmenos2$foto_mes_formato_fecha,sep='-')

   dataset[, codigo := paste(numero_de_cliente, foto_mes_formato_fecha, sep='-') ]

   dataset[ auxiliarmenos1,
            on = "codigo",
            transaccionesmenos1 := i.cantidad_total_transacciones ]

   dataset[ auxiliarmenos2,
            on = "codigo",
            transaccionesmenos2 := i.cantidad_total_transacciones ]

   dataset[, cantidad_total_transacciones_quarter := rowSums(cbind(cantidad_total_transacciones +
    transaccionesmenos1 + transaccionesmenos2),na.rm=T) ]

   dataset[, codigo := NULL ]
   dataset[, transaccionesmenos1 := NULL ]
   dataset[, transaccionesmenos2 := NULL ]
   dataset[, foto_mes_formato_fecha := NULL ]
   rm(auxiliarmenos1)
   rm(auxiliarmenos2)
  }

  if( atributos_presentes( c("cantidad_total_transacciones_quarter") ))
    dataset[, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 5]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 2]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 3, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 1.2]

  # variable extraida de una tesis de maestria de Irlanda
  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # se crean los nuevos campos para MasterCard  y Visa,
  #  teniendo en cuenta los NA's
  # varias formas de combinar Visa_status y Master_status
  if( atributos_presentes( c("Master_status", "Visa_status") ))
  {
    dataset[, vm_status01 := pmax(Master_status, Visa_status, na.rm = TRUE)]
    dataset[, vm_status02 := Master_status + Visa_status]

    dataset[, vm_status03 := pmax(
      ifelse(is.na(Master_status), 10, Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status)
    )]

    dataset[, vm_status04 := ifelse(is.na(Master_status), 10, Master_status)
      + ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status05 := ifelse(is.na(Master_status), 10, Master_status)
      + 100 * ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status06 := ifelse(is.na(Visa_status),
      ifelse(is.na(Master_status), 10, Master_status),
      Visa_status
    )]

    dataset[, mv_status07 := ifelse(is.na(Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status),
      Master_status
    )]
  }


  # combino MasterCard y Visa
  if( atributos_presentes( c("Master_mfinanciacion_limite", "Visa_mfinanciacion_limite") ))
    dataset[, vm_mfinanciacion_limite := rowSums(cbind(Master_mfinanciacion_limite, Visa_mfinanciacion_limite), na.rm = TRUE)]

  if( atributos_presentes( c("Master_Fvencimiento", "Visa_Fvencimiento") ))
    dataset[, vm_Fvencimiento := pmin(Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE)]

  if( atributos_presentes( c("Master_Finiciomora", "Visa_Finiciomora") ))
    dataset[, vm_Finiciomora := pmin(Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldototal", "Visa_msaldototal") ))
    dataset[, vm_msaldototal := rowSums(cbind(Master_msaldototal, Visa_msaldototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldopesos", "Visa_msaldopesos") ))
    dataset[, vm_msaldopesos := rowSums(cbind(Master_msaldopesos, Visa_msaldopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldodolares", "Visa_msaldodolares") ))
    dataset[, vm_msaldodolares := rowSums(cbind(Master_msaldodolares, Visa_msaldodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumospesos", "Visa_mconsumospesos") ))
    dataset[, vm_mconsumospesos := rowSums(cbind(Master_mconsumospesos, Visa_mconsumospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumosdolares", "Visa_mconsumosdolares") ))
    dataset[, vm_mconsumosdolares := rowSums(cbind(Master_mconsumosdolares, Visa_mconsumosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mlimitecompra", "Visa_mlimitecompra") ))
    dataset[, vm_mlimitecompra := rowSums(cbind(Master_mlimitecompra, Visa_mlimitecompra), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantopesos", "Visa_madelantopesos") ))
    dataset[, vm_madelantopesos := rowSums(cbind(Master_madelantopesos, Visa_madelantopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantodolares", "Visa_madelantodolares") ))
    dataset[, vm_madelantodolares := rowSums(cbind(Master_madelantodolares, Visa_madelantodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fultimo_cierre", "Visa_fultimo_cierre") ))
    dataset[, vm_fultimo_cierre := pmax(Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagado", "Visa_mpagado") ))
    dataset[, vm_mpagado := rowSums(cbind(Master_mpagado, Visa_mpagado), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagospesos", "Visa_mpagospesos") ))
    dataset[, vm_mpagospesos := rowSums(cbind(Master_mpagospesos, Visa_mpagospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagosdolares", "Visa_mpagosdolares") ))
    dataset[, vm_mpagosdolares := rowSums(cbind(Master_mpagosdolares, Visa_mpagosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fechaalta", "Visa_fechaalta") ))
    dataset[, vm_fechaalta := pmax(Master_fechaalta, Visa_fechaalta, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumototal", "Visa_mconsumototal") ))
    dataset[, vm_mconsumototal := rowSums(cbind(Master_mconsumototal, Visa_mconsumototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cconsumos", "Visa_cconsumos") ))
    dataset[, vm_cconsumos := rowSums(cbind(Master_cconsumos, Visa_cconsumos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cadelantosefectivo", "Visa_cadelantosefectivo") ))
    dataset[, vm_cadelantosefectivo := rowSums(cbind(Master_cadelantosefectivo, Visa_cadelantosefectivo), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagominimo", "Visa_mpagominimo") ))
    dataset[, vm_mpagominimo := rowSums(cbind(Master_mpagominimo, Visa_mpagominimo), na.rm = TRUE)]

  # a partir de aqui juego con la suma de Mastercard y Visa
  if( atributos_presentes( c("Master_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Master_mlimitecompra := Master_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("Visa_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Visa_mlimitecompra := Visa_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldototal", "vm_mlimitecompra") ))
    dataset[, vmr_msaldototal := vm_msaldototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_mlimitecompra") ))
    dataset[, vmr_msaldopesos := vm_msaldopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_msaldototal") ))
    dataset[, vmr_msaldopesos2 := vm_msaldopesos / vm_msaldototal]

  if( atributos_presentes( c("vm_msaldodolares", "vm_mlimitecompra") ))
    dataset[, vmr_msaldodolares := vm_msaldodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldodolares", "vm_msaldototal") ))
    dataset[, vmr_msaldodolares2 := vm_msaldodolares / vm_msaldototal]

  if( atributos_presentes( c("vm_mconsumospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumospesos := vm_mconsumospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumosdolares := vm_mconsumosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantopesos", "vm_mlimitecompra") ))
    dataset[, vmr_madelantopesos := vm_madelantopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantodolares", "vm_mlimitecompra") ))
    dataset[, vmr_madelantodolares := vm_madelantodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagado", "vm_mlimitecompra") ))
    dataset[, vmr_mpagado := vm_mpagado / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mpagospesos := vm_mpagospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mpagosdolares := vm_mpagosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumototal", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumototal := vm_mconsumototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagominimo", "vm_mlimitecompra") ))
    dataset[, vmr_mpagominimo := vm_mpagominimo / vm_mlimitecompra]

  # Aqui debe usted agregar sus propias nuevas variables

  # valvula de seguridad para evitar valores infinitos
  # paso los infinitos a NULOS
  infinitos <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
        add_log(paste0(
      "ATENCION, hay ", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n")
    )

    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    add_log(
      paste0("ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    ))

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    add_log("Si no te gusta la decision, modifica a gusto el programa!\n\n")


    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat( "fin AgregarVariables_IntraMes()\n")
  add_log( "fin AgregarVariables_IntraMes()\n")
}


In [40]:
# agrego las variables intra mes
AgregarVariables_IntraMes(dataset)

inicio AgregarVariables_IntraMes()
ATENCION, hay 8 valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0
Si no te gusta la decision, modifica a gusto el programa!

fin AgregarVariables_IntraMes()


In [41]:
# visualizo las columas del dataset a esta etapa
ncol(dataset)
colnames(dataset)

add_log(ncol(dataset))

[1] 45

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "mpayroll_sobre_edad"        
[35] "ctrx_quarter_normalizado"    "foto_mes_formato_fecha"     
[37] "vm_status01"                 "vm_status02"                
[39] "vm_status03"                 "vm_status04"                
[41] "vm_status05"                 "vm_status06"                
[43] "mv_status07"                 "vm_fechaalta"               
[45] "vm_mpagominimo"

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [42]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [43]:
ncol(dataset)
colnames(dataset)

[1] 213

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "kmes"                               "mpayroll_sobre_edad"               
 [35] "ctrx_quarter_normalizado"           "foto_mes_formato_fecha"            
 [37] "vm_status01"                        "vm_status02"                       
 [39] "vm_status03"                        "vm_status04"                       
 [41] "vm_status05"                        "vm_status06"                       
 [43] "mv_status07"                        "vm_fechaalta"                      
 [45] "vm_mpagominimo"                     "internet_lag1"                     
 [47] "cliente_edad_lag1"                  "cliente_antiguedad_lag1"           
 [49] "mrentabilidad_lag1"                 "mrentabilidad_annual_lag1"         
 [51] "mcomisiones_lag1"                   "mactivos_margen_lag1"              
 [53] "mpasivos_margen_lag1"               "cproductos_lag1"                   
 [55] "mcuenta_corriente_lag1"             "mcaja_ahorro_lag1"                 
 [57] "cdescubierto_preacordado_lag1"      "mcuentas_saldo_lag1"               
 [59] "ctarjeta_visa_transacciones_lag1"   "mtarjeta_visa_consumo_lag1"        
 [61] "mtarjeta_master_consumo_lag1"       "mprestamos_personales_lag1"        
 [63] "cpayroll_trx_lag1"                  "mpayroll_lag1"                     
 [65] "ccomisiones_mantenimiento_lag1"     "ccallcenter_transacciones_lag1"    
 [67] "chomebanking_transacciones_lag1"    "ctrx_quarter_lag1"                 
 [69] "Master_status_lag1"                 "Master_fechaalta_lag1"             
 [71] "Master_mpagominimo_lag1"            "Visa_status_lag1"                  
 [73] "Visa_fechaalta_lag1"                "Visa_mpagominimo_lag1"             
 [75] "kmes_lag1"                          "mpayroll_sobre_edad_lag1"          
 [77] "ctrx_quarter_normalizado_lag1"      "foto_mes_formato_fecha_lag1"       
 [79] "vm_status01_lag1"                   "vm_status02_lag1"                  
 [81] "vm_status03_lag1"                   "vm_status04_lag1"                  
 [83] "vm_status05_lag1"                   "vm_status06_lag1"                  
 [85] "mv_status07_lag1"                   "vm_fechaalta_lag1"                 
 [87] "vm_mpagominimo_lag1"                "internet_lag2"                     
 [89] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [91] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [93] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [95] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [97] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [99] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
[1

#### 9.7.1.5  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest



In [44]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm



In [63]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  add_log( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [65]:
AgregaVarRandomForest()

inicio AgregaVarRandomForest()


ERROR: Error in data$construct(): Check failed: (num_data) > (0) at io/dataset.cpp, line 39 .




#### 9.7.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr*

El objetivo de esta etapa NO es mejorar el modelo predictivo

El objetivo es eliminar campos poco importantes para hacer espacio a nuevos campos, debido a las restricciones de memoria RAM.

In [46]:
VPOS_CORTE <- c()

fganancia_lgbm_meseta <- function(probs, datos) {
  vlabels <- get_field(datos, "label")
  vpesos <- get_field(datos, "weight")

  tbl <- as.data.table(list(
    "prob" = probs,
    "gan" = ifelse(vlabels == 1 & vpesos > 1, PARAM$CN$train$gan1, PARAM$CN$train$gan0)
  ))

  setorder(tbl, -prob)
  tbl[, posicion := .I]
  tbl[, gan_acum := cumsum(gan)]
  setorder(tbl, -gan_acum) # voy por la meseta

  gan <- mean(tbl[1:500, gan_acum]) # meseta de tamaño 500

  pos_meseta <- tbl[1:500, median(posicion)]
  VPOS_CORTE <<- c(VPOS_CORTE, pos_meseta)

  return(list(
    "name" = "ganancia",
    "value" = gan,
    "higher_better" = TRUE
  ))
}


In [47]:
# Elimina del dataset las variables que estan por debajo
#  de la capa geologica de canaritos
# se llama varias veces, luego de agregar muchas variables nuevas,
#  para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1

campitos <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

CanaritosAsesinos <- function(
  canaritos_ratio,
  canaritos_desvios,
  canaritos_semilla) {

  cat( "inicio CanaritosAsesinos()\n")
  add_log( "inicio CanaritosAsesinos()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$CN$train$clase01_valor1,
      clase01 := 1L ]

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  for (i in 1:(ncol(dataset) * canaritos_ratio)) {
    dataset[, paste0("canarito", i) := runif(nrow(dataset))]
  }

  campos_buenos <- setdiff(
    colnames(dataset),
    c( campitos, "clase01")
  )

  azar <- runif(nrow(dataset))

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$CN$train$training &
      (clase01 == 1 | azar < PARAM$CN$train$undersampling))]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    weight = dataset[
      entrenamiento == TRUE,
      ifelse(clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )

  dvalid <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% PARAM$CN$train$validation, campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$CN$train$validation, clase01],
    weight = dataset[
      foto_mes %in% PARAM$CN$train$validation,
      ifelse( clase_ternaria %in% PARAM$CN$train$positivos, 1.0000001, 1.0)
    ],
    free_raw_data = FALSE
  )


  param <- list(
    objective = "binary",
    metric = "custom",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    seed = canaritos_semilla,
    max_depth = -1, # -1 significa no limitar,  por ahora lo dejo fijo
    min_gain_to_split = 0.0, # por ahora, lo dejo fijo
    lambda_l1 = 0.0, # por ahora, lo dejo fijo
    lambda_l2 = 0.0, # por ahora, lo dejo fijo
    max_bin = 31, # por ahora, lo dejo fijo
    num_iterations = 9999, # un numero grande, lo limita early_stopping_rounds
    force_row_wise = TRUE, # para que los alumnos no se atemoricen con  warning
    learning_rate = 0.065,
    feature_fraction = 1.0, # lo seteo en 1
    min_data_in_leaf = 260,
    num_leaves = 60,
    early_stopping_rounds = 200,
    num_threads = 1
  )

  set.seed(canaritos_semilla, kind = "L'Ecuyer-CMRG")
  modelo <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalid),
    eval = fganancia_lgbm_meseta,
    param = param,
    verbose = -100
  )

  tb_importancia <- lgb.importance(model = modelo)
  tb_importancia[, pos := .I]

  fwrite(tb_importancia,
    file = paste0("impo_", GVEZ, ".txt"),
    sep = "\t"
  )

  GVEZ <<- GVEZ + 1

  umbral <- tb_importancia[
    Feature %like% "canarito",
    median(pos) + canaritos_desvios * sd(pos)
  ] # Atencion corto en la mediana mas desvios!!

  col_utiles <- tb_importancia[
    pos < umbral & !(Feature %like% "canarito"),
    Feature
  ]

  col_utiles <- unique(c(
    col_utiles,
    c(campitos, "mes")
  ))

  col_inutiles <- setdiff(colnames(dataset), col_utiles)

  dataset[, (col_inutiles) := NULL]

  cat( "fin CanaritosAsesinos()\n")
  add_log( "fin CanaritosAsesinos()\n")

  return( tb_importancia )
}


In [48]:
# Estos DOS parametros son los que se deben modificar
PARAM$CN$ratio <- 0.2
PARAM$CN$desvios <- 2


# Parametros quasi fijos
# Parametros de un LightGBM que se genera para estimar la column importance
PARAM$CN$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$CN$train$positivos <- c( "BAJA+2")
PARAM$CN$train$training <- c( 202101, 202102, 202103)
PARAM$CN$train$validation <- c( 202105 )
PARAM$CN$train$undersampling <- 0.1
PARAM$CN$train$gan1 <- 117000
PARAM$CN$train$gan0 <-  -3000

In [49]:
# la llamada a Canaritos Asesinos
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)


inicio CanaritosAsesinos()
fin CanaritosAsesinos()


In [50]:
# grabo la importancia, ver el archivo directamente en la carpeta

fwrite( tb_importancia,
  file="canaritos.txt",
  sep="\t"
)

In [51]:
# verifico
ncol(dataset)
colnames(dataset)

add_log(ncol(dataset))

[1] 76

[1] "numero_de_cliente"                 "foto_mes"                         
 [3] "cliente_edad"                      "mrentabilidad"                    
 [5] "mrentabilidad_annual"              "mcomisiones"                      
 [7] "mactivos_margen"                   "mpasivos_margen"                  
 [9] "cproductos"                        "mcuenta_corriente"                
[11] "mcaja_ahorro"                      "cdescubierto_preacordado"         
[13] "mcuentas_saldo"                    "ctarjeta_visa_transacciones"      
[15] "mtarjeta_visa_consumo"             "mtarjeta_master_consumo"          
[17] "mprestamos_personales"             "cpayroll_trx"                     
[19] "ccomisiones_mantenimiento"         "chomebanking_transacciones"       
[21] "Master_fechaalta"                  "Master_mpagominimo"               
[23] "Visa_fechaalta"                    "Visa_mpagominimo"                 
[25] "clase_ternaria"                    "mpayroll_sobre_edad"              
[27] "ctrx_quarter_normalizado"          "vm_mpagominimo"                   
[29] "cliente_edad_lag1"                 "mrentabilidad_lag1"               
[31] "mrentabilidad_annual_lag1"         "mcomisiones_lag1"                 
[33] "mpasivos_margen_lag1"              "mcuenta_corriente_lag1"           
[35] "mcaja_ahorro_lag1"                 "mcuentas_saldo_lag1"              
[37] "chomebanking_transacciones_lag1"   "Master_fechaalta_lag1"            
[39] "Visa_fechaalta_lag1"               "Visa_mpagominimo_lag1"            
[41] "ctrx_quarter_normalizado_lag1"     "vm_fechaalta_lag1"                
[43] "vm_mpagominimo_lag1"               "cliente_edad_lag2"                
[45] "cliente_antiguedad_lag2"           "mrentabilidad_lag2"               
[47] "mrentabilidad_annual_lag2"         "mactivos_margen_lag2"             
[49] "mpasivos_margen_lag2"              "mcuenta_corriente_lag2"           
[51] "mcaja_ahorro_lag2"                 "mprestamos_personales_lag2"       
[53] "mpayroll_lag2"                     "chomebanking_transacciones_lag2"  
[55] "ctrx_quarter_lag2"                 "Master_fechaalta_lag2"            
[57] "Visa_fechaalta_lag2"               "Visa_mpagominimo_lag2"            
[59] "mrentabilidad_delta2"              "mrentabilidad_annual_delta1"      
[61] "mrentabilidad_annual_delta2"       "mcomisiones_delta2"               
[63] "mactivos_margen_delta1"            "mactivos_margen_delta2"           
[65] "cproductos_delta1"                 "mcuentas_saldo_delta1"            
[67] "mtarjeta_visa_consumo_delta2"      "ccomisiones_mantenimiento_delta1" 
[69] "ccomisiones_mantenimiento_delta2"  "ccallcenter_transacciones_delta2" 
[71] "chomebanking_transacciones_delta1" "chomebanking_transacciones_delta2"
[73] "ctrx_quarter_delta1"               "ctrx_quarter_delta2"              
[75] "Visa_mpagominimo_delta2"           "vm_mpagominimo_delta1"

In [52]:
# Install and load required packages
if (!require("GA")) install.packages("GA")
if (!require("dplyr")) install.packages("dplyr")
library(GA)
library(dplyr)

Loading required package: GA

Loading required package: foreach

Loading required package: iterators

Package 'GA' version 3.2.4
Type 'citation("GA")' for citing this R package in publications.


Attaching package: ‘GA’


The following object is masked from ‘package:utils’:

    de


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [53]:
campos_a_omitir = c('numero_de_cliente','foto_mes','azar')

In [54]:
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                "mrentabilidad_annual"        
 [3] "mcomisiones"                  "mactivos_margen"             
 [5] "mpasivos_margen"              "mcuenta_corriente"           
 [7] "mcaja_ahorro"                 "mcuentas_saldo"              
 [9] "mtarjeta_visa_consumo"        "mtarjeta_master_consumo"     
[11] "mprestamos_personales"        "Master_mpagominimo"          
[13] "Visa_mpagominimo"             "mpayroll_sobre_edad"         
[15] "vm_mpagominimo"               "mrentabilidad_lag1"          
[17] "mrentabilidad_annual_lag1"    "mcomisiones_lag1"            
[19] "mpasivos_margen_lag1"         "mcuenta_corriente_lag1"      
[21] "mcaja_ahorro_lag1"            "mcuentas_saldo_lag1"         
[23] "Visa_mpagominimo_lag1"        "vm_mpagominimo_lag1"         
[25] "mrentabilidad_lag2"           "mrentabilidad_annual_lag2"   
[27] "mactivos_margen_lag2"         "mpasivos_margen_lag2"        
[29] "mcuenta_corriente_lag2"       "mcaja_ahorro_lag2"           
[31] "mprestamos_personales_lag2"   "mpayroll_lag2"               
[33] "Visa_mpagominimo_lag2"        "mrentabilidad_delta2"        
[35] "mrentabilidad_annual_delta1"  "mrentabilidad_annual_delta2" 
[37] "mcomisiones_delta2"           "mactivos_margen_delta1"      
[39] "mactivos_margen_delta2"       "mcuentas_saldo_delta1"       
[41] "mtarjeta_visa_consumo_delta2" "Visa_mpagominimo_delta2"     
[43] "vm_mpagominimo_delta1"

In [55]:
campos_rf <- colnames(dataset)
campos_rf <- campos_rf[campos_rf %like%
  "^rf"]
campos_rf

character(0)

In [56]:
campos_lag <- colnames(dataset)
campos_lag <- campos_lag[campos_lag %like%
  "(lag|delta).$"]
campos_lag <- campos_lag[!campos_lag %in% campos_monetarios]
campos_lag

[1] "cliente_edad_lag1"                 "chomebanking_transacciones_lag1"  
 [3] "Master_fechaalta_lag1"             "Visa_fechaalta_lag1"              
 [5] "ctrx_quarter_normalizado_lag1"     "vm_fechaalta_lag1"                
 [7] "cliente_edad_lag2"                 "cliente_antiguedad_lag2"          
 [9] "chomebanking_transacciones_lag2"   "ctrx_quarter_lag2"                
[11] "Master_fechaalta_lag2"             "Visa_fechaalta_lag2"              
[13] "cproductos_delta1"                 "ccomisiones_mantenimiento_delta1" 
[15] "ccomisiones_mantenimiento_delta2"  "ccallcenter_transacciones_delta2" 
[17] "chomebanking_transacciones_delta1" "chomebanking_transacciones_delta2"
[19] "ctrx_quarter_delta1"               "ctrx_quarter_delta2"

In [57]:
is_bool_column <- function(columname) {
    tryCatch({
      if (class(max(dataset[[columname]])) %in% c('numeric', 'integer')){
      maxx = max(dataset[[columname]])
      minn = min(dataset[[columname]])
      return ((maxx - minn) == 1)
} else{
          return (FALSE)
  }
}, error = function(e){
        FALSE
})}

get_non_bool_cols <- function(dataset) {
  all_cols <- names(dataset)
  bool_cols <- sapply(all_cols, function(col) is_bool_column(dataset, col))
  return(all_cols[!bool_cols])
}


In [58]:
campos_int <- colnames(dataset)
campos_int <- campos_int[!campos_int %in% c(campos_a_omitir, campos_monetarios, campos_lag, campos_rf)]
campos_int <- campos_int[!sapply(campos_int,is_bool_column)]
campos_int = na.omit(campos_int)


In [59]:
campos_lag
campos_int
campos_rf
campos_monetarios

[1] "cliente_edad_lag1"                 "chomebanking_transacciones_lag1"  
 [3] "Master_fechaalta_lag1"             "Visa_fechaalta_lag1"              
 [5] "ctrx_quarter_normalizado_lag1"     "vm_fechaalta_lag1"                
 [7] "cliente_edad_lag2"                 "cliente_antiguedad_lag2"          
 [9] "chomebanking_transacciones_lag2"   "ctrx_quarter_lag2"                
[11] "Master_fechaalta_lag2"             "Visa_fechaalta_lag2"              
[13] "cproductos_delta1"                 "ccomisiones_mantenimiento_delta1" 
[15] "ccomisiones_mantenimiento_delta2"  "ccallcenter_transacciones_delta2" 
[17] "chomebanking_transacciones_delta1" "chomebanking_transacciones_delta2"
[19] "ctrx_quarter_delta1"               "ctrx_quarter_delta2"

[1] "cliente_edad"              "cproductos"               
[3] "cpayroll_trx"              "ccomisiones_mantenimiento"
[5] "clase_ternaria"            "ctrx_quarter_normalizado" 
attr(,"na.action")
[1] 3 6 7 8
attr(,"class")
[1] "omit"

character(0)

[1] "mrentabilidad"                "mrentabilidad_annual"        
 [3] "mcomisiones"                  "mactivos_margen"             
 [5] "mpasivos_margen"              "mcuenta_corriente"           
 [7] "mcaja_ahorro"                 "mcuentas_saldo"              
 [9] "mtarjeta_visa_consumo"        "mtarjeta_master_consumo"     
[11] "mprestamos_personales"        "Master_mpagominimo"          
[13] "Visa_mpagominimo"             "mpayroll_sobre_edad"         
[15] "vm_mpagominimo"               "mrentabilidad_lag1"          
[17] "mrentabilidad_annual_lag1"    "mcomisiones_lag1"            
[19] "mpasivos_margen_lag1"         "mcuenta_corriente_lag1"      
[21] "mcaja_ahorro_lag1"            "mcuentas_saldo_lag1"         
[23] "Visa_mpagominimo_lag1"        "vm_mpagominimo_lag1"         
[25] "mrentabilidad_lag2"           "mrentabilidad_annual_lag2"   
[27] "mactivos_margen_lag2"         "mpasivos_margen_lag2"        
[29] "mcuenta_corriente_lag2"       "mcaja_ahorro_lag2"           
[31] "mprestamos_personales_lag2"   "mpayroll_lag2"               
[33] "Visa_mpagominimo_lag2"        "mrentabilidad_delta2"        
[35] "mrentabilidad_annual_delta1"  "mrentabilidad_annual_delta2" 
[37] "mcomisiones_delta2"           "mactivos_margen_delta1"      
[39] "mactivos_margen_delta2"       "mcuentas_saldo_delta1"       
[41] "mtarjeta_visa_consumo_delta2" "Visa_mpagominimo_delta2"     
[43] "vm_mpagominimo_delta1"

In [60]:
colnames(dataset)

[1] "numero_de_cliente"                 "foto_mes"                         
 [3] "cliente_edad"                      "mrentabilidad"                    
 [5] "mrentabilidad_annual"              "mcomisiones"                      
 [7] "mactivos_margen"                   "mpasivos_margen"                  
 [9] "cproductos"                        "mcuenta_corriente"                
[11] "mcaja_ahorro"                      "cdescubierto_preacordado"         
[13] "mcuentas_saldo"                    "ctarjeta_visa_transacciones"      
[15] "mtarjeta_visa_consumo"             "mtarjeta_master_consumo"          
[17] "mprestamos_personales"             "cpayroll_trx"                     
[19] "ccomisiones_mantenimiento"         "chomebanking_transacciones"       
[21] "Master_fechaalta"                  "Master_mpagominimo"               
[23] "Visa_fechaalta"                    "Visa_mpagominimo"                 
[25] "clase_ternaria"                    "mpayroll_sobre_edad"              
[27] "ctrx_quarter_normalizado"          "vm_mpagominimo"                   
[29] "cliente_edad_lag1"                 "mrentabilidad_lag1"               
[31] "mrentabilidad_annual_lag1"         "mcomisiones_lag1"                 
[33] "mpasivos_margen_lag1"              "mcuenta_corriente_lag1"           
[35] "mcaja_ahorro_lag1"                 "mcuentas_saldo_lag1"              
[37] "chomebanking_transacciones_lag1"   "Master_fechaalta_lag1"            
[39] "Visa_fechaalta_lag1"               "Visa_mpagominimo_lag1"            
[41] "ctrx_quarter_normalizado_lag1"     "vm_fechaalta_lag1"                
[43] "vm_mpagominimo_lag1"               "cliente_edad_lag2"                
[45] "cliente_antiguedad_lag2"           "mrentabilidad_lag2"               
[47] "mrentabilidad_annual_lag2"         "mactivos_margen_lag2"             
[49] "mpasivos_margen_lag2"              "mcuenta_corriente_lag2"           
[51] "mcaja_ahorro_lag2"                 "mprestamos_personales_lag2"       
[53] "mpayroll_lag2"                     "chomebanking_transacciones_lag2"  
[55] "ctrx_quarter_lag2"                 "Master_fechaalta_lag2"            
[57] "Visa_fechaalta_lag2"               "Visa_mpagominimo_lag2"            
[59] "mrentabilidad_delta2"              "mrentabilidad_annual_delta1"      
[61] "mrentabilidad_annual_delta2"       "mcomisiones_delta2"               
[63] "mactivos_margen_delta1"            "mactivos_margen_delta2"           
[65] "cproductos_delta1"                 "mcuentas_saldo_delta1"            
[67] "mtarjeta_visa_consumo_delta2"      "ccomisiones_mantenimiento_delta1" 
[69] "ccomisiones_mantenimiento_delta2"  "ccallcenter_transacciones_delta2" 
[71] "chomebanking_transacciones_delta1" "chomebanking_transacciones_delta2"
[73] "ctrx_quarter_delta1"               "ctrx_quarter_delta2"              
[75] "Visa_mpagominimo_delta2"           "vm_mpagominimo_delta1"

# Genetic Algorithm

## GA Test

In [61]:
if( !require("gramEvol")) install.packages("gramEvol")
if( !require("GA")) install.packages("GA")

Loading required package: gramEvol



In [62]:
genetic_dataset = copy(dataset)
head(genetic_dataset)

numero_de_cliente,foto_mes,cliente_edad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,mcuenta_corriente,⋯,mtarjeta_visa_consumo_delta2,ccomisiones_mantenimiento_delta1,ccomisiones_mantenimiento_delta2,ccallcenter_transacciones_delta2,chomebanking_transacciones_delta1,chomebanking_transacciones_delta2,ctrx_quarter_delta1,ctrx_quarter_delta2,Visa_mpagominimo_delta2,vm_mpagominimo_delta1
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
29186441,202005,61,2853.880,53033.33,2038.534,-168.4741,863.5782,7,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29186441,202006,61,NA,NA,NA,NA,NA,7,0,⋯,NA,0,NA,NA,NA,NA,-15,NA,NA,-247.790951
29186441,202007,61,3052.337,53831.51,1970.036,-217.1336,1139.8112,7,0,⋯,-734.8005,0,0,0,NA,21,18,3,-71.77407,176.016884
29186441,202008,61,4120.270,55020.54,1921.690,-228.3395,2102.6622,7,0,⋯,NA,0,0,NA,-23,NA,2,20,170.19916,-5.817727
29186441,202009,61,4586.567,55348.24,1962.288,-236.0629,2473.2988,7,0,⋯,2080.3284,0,0,0,0,-23,-1,1,104.61587,110.433600
29186441,202010,61,5379.173,56610.47,1895.436,-220.9813,3190.9252,7,0,⋯,1247.2643,0,0,0,24,24,5,4,228.20554,117.771945


In [68]:
safeEval <- function(expr) {
  val <- try(eval(parse(text = expr), envir = dataset), silent = TRUE)
  #if (inherits(val, "try-error") || any(is.nan(val)) || any(is.infinite(val))) {
  #  return(rep(NA, nrow(df)))
  #}
  val
}


In [69]:
# Grammar definition
grammarDef <- CreateGrammar(list(
  expr = gsrule("genetic_dataset[,<m_var>]+genetic_dataset[,<m_var>]|genetic_dataset[,<m_var>]/genetic_dataset[,<i_var>]|genetic_dataset[,<i_var>]*genetic_dataset[,<i_var>]"),
  m_var  = gvrule(campos_monetarios),
  i_var  = gvrule(campos_int)
))

# Function to generate a new column from grammar
generate_column <- function() {
  expr_str <- GrammarRandomExpression(grammarDef, startSymb = GrammarStartSymbol(grammarDef))
  # Evaluate the generated expression
  val <- safeEval(expr_str)
  return(val)
}

# Number of new features
gen_columns <- 500
for (i in 1:gen_columns) {

  new_col <- generate_column()
  genetic_dataset[, paste0("gen_", i):= new_col]
}

head(genetic_dataset)

numero_de_cliente,foto_mes,cliente_edad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,mcuenta_corriente,⋯,gen_491,gen_492,gen_493,gen_494,gen_495,gen_496,gen_497,gen_498,gen_499,gen_500
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<try-errr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
29186441,202005,61,2853.880,53033.33,2038.534,-168.4741,863.5782,7,0,⋯,NA,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",NA,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202006,61,NA,NA,NA,NA,NA,7,0,⋯,TRUE,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202007,61,3052.337,53831.51,1970.036,-217.1336,1139.8112,7,0,⋯,NA,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",NA,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202008,61,4120.270,55020.54,1921.690,-228.3395,2102.6622,7,0,⋯,TRUE,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202009,61,4586.567,55348.24,1962.288,-236.0629,2473.2988,7,0,⋯,TRUE,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202010,61,5379.173,56610.47,1895.436,-220.9813,3190.9252,7,0,⋯,TRUE,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE


In [70]:

infinitos <- lapply(
    names(genetic_dataset),
    function(.name) genetic_dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
        add_log(paste0(
      "ATENCION, hay ", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n")
    )

    genetic_dataset[mapply(is.infinite, genetic_dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(genetic_dataset),
    function(.name) genetic_dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    add_log(
      paste0("ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    ))

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    add_log("Si no te gusta la decision, modifica a gusto el programa!\n\n")


    genetic_dataset[mapply(is.nan, genetic_dataset)] <<- 0
      }

In [71]:
dataset_aux = copy(dataset)
dataset = copy(genetic_dataset)
head(dataset)

numero_de_cliente,foto_mes,cliente_edad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,mcuenta_corriente,⋯,gen_491,gen_492,gen_493,gen_494,gen_495,gen_496,gen_497,gen_498,gen_499,gen_500
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<try-errr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
29186441,202005,61,2853.880,53033.33,2038.534,-168.4741,863.5782,7,0,⋯,NA,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",NA,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202006,61,NA,NA,NA,NA,NA,7,0,⋯,TRUE,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202007,61,3052.337,53831.51,1970.036,-217.1336,1139.8112,7,0,⋯,NA,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",NA,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202008,61,4120.270,55020.54,1921.690,-228.3395,2102.6622,7,0,⋯,TRUE,NA,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202009,61,4586.567,55348.24,1962.288,-236.0629,2473.2988,7,0,⋯,TRUE,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
29186441,202010,61,5379.173,56610.47,1895.436,-220.9813,3190.9252,7,0,⋯,TRUE,TRUE,TRUE,"Error in FUN(left, right) : non-numeric argument to binary operator",TRUE,TRUE,TRUE,TRUE,TRUE,TRUE


In [72]:
tb_importancia <- CanaritosAsesinos(
  canaritos_ratio = PARAM$CN$ratio,
  canaritos_desvios = PARAM$CN$desvios,
  canaritos_semilla = PARAM$semilla_primigenia
)


inicio CanaritosAsesinos()
fin CanaritosAsesinos()


In [73]:
tb_importancia


Feature,Gain,Cover,Frequency,pos
<chr>,<dbl>,<dbl>,<dbl>,<int>
ctrx_quarter_normalizado,0.281731414,0.037014751,0.021264689,1
mpayroll_sobre_edad,0.044695240,0.040871757,0.024062675,2
mtarjeta_visa_consumo,0.041476214,0.025763695,0.020145495,3
mcuentas_saldo,0.025590041,0.019167752,0.017907107,4
mprestamos_personales,0.023081248,0.030009229,0.013989927,5
mcaja_ahorro_lag2,0.022517659,0.013424343,0.012870733,6
ccomisiones_mantenimiento_delta2,0.021828936,0.018633075,0.008393956,7
cpayroll_trx,0.018074427,0.007306540,0.003357583,8
mcaja_ahorro,0.017958242,0.003798667,0.005036374,9


In [74]:
head(dataset)

numero_de_cliente,foto_mes,cliente_edad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,mcuenta_corriente,⋯,ctrx_quarter_delta2,Visa_mpagominimo_delta2,vm_mpagominimo_delta1,gen_122,gen_167,gen_240,gen_367,gen_402,gen_457,gen_470
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<int>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
29186441,202005,61,2853.880,53033.33,2038.534,-168.4741,863.5782,7,0,⋯,NA,NA,NA,NA,TRUE,TRUE,TRUE,NA,NA,TRUE
29186441,202006,61,NA,NA,NA,NA,NA,7,0,⋯,NA,NA,-247.790951,TRUE,TRUE,TRUE,TRUE,TRUE,NA,TRUE
29186441,202007,61,3052.337,53831.51,1970.036,-217.1336,1139.8112,7,0,⋯,3,-71.77407,176.016884,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE
29186441,202008,61,4120.270,55020.54,1921.690,-228.3395,2102.6622,7,0,⋯,20,170.19916,-5.817727,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE
29186441,202009,61,4586.567,55348.24,1962.288,-236.0629,2473.2988,7,0,⋯,1,104.61587,110.433600,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE
29186441,202010,61,5379.173,56610.47,1895.436,-220.9813,3190.9252,7,0,⋯,4,228.20554,117.771945,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE


### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    r

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 30

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  #learning_rate= 0.03,
  #feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 1500L, upper = 2500L),
  makeIntegerParam("min_data_in_leaf", lower = 11, upper = 250L),
  makeNumericParam("learning_rate", lower = 0.001, upper = .3),
  makeNumericParam("feature_fraction", lower = 0.3, upper = .8)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Wed Aug 13 02:41:28 AM 2025 AUC 0.930863179163027

Wed Aug 13 02:43:46 AM 2025 AUC 0.930148825591607

Wed Aug 13 02:45:51 AM 2025 AUC 0.926955667444431

Wed Aug 13 02:48:25 AM 2025 AUC 0.925496374939466

Wed Aug 13 02:51:09 AM 2025 AUC 0.918303485572746

Wed Aug 13 02:54:00 AM 2025 AUC 0.924197423878363

Wed Aug 13 02:56:12 AM 2025 AUC 0.92363831419045

Wed Aug 13 02:57:30 AM 2025 AUC 0.926684338209227

Wed Aug 13 03:01:40 AM 2025 AUC 0.929614276876308

Wed Aug 13 03:04:07 AM 2025 AUC 0.914274095820232

Wed Aug 13 03:14:18 AM 2025 AUC 0.931621325412033

Wed Aug 13 03:17:14 AM 2025 AUC 0.940823580387368

Wed Aug 13 03:19:28 AM 2025 AUC 0.903860243431678

Wed Aug 13 03:23:03 AM 2025 AUC 0.933063239870337

Wed Aug 13 03:28:28 AM 2025 AUC 0.940026507313841

Wed Aug 13 03:30:09 AM 2025 AUC 0.920925021838412

[mbo] 0: num_leaves=2271; min_data_in_leaf=91; learning_rate=0.077; feature_fraction=0.511 : y = 0.931 : 218.4 secs : initdesign

[mbo] 

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:       1655               87            940


### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "data-mining-gerencial-2025-a"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )
    cat(mensaje)
  linea <- paste( comando, competencia, arch, mensaje)
  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

-m 'envios=800  semilla=424243'

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c data-mining-gerencial-2025-a -f ./kaggle/KA6300_generencia_FE_gen_testB2_800.csv -m 'envios=800  semilla=424243'' had status 1”


 
-m 'envios=850  semilla=424243'

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c data-mining-gerencial-2025-a -f ./kaggle/KA6300_generencia_FE_gen_testB2_850.csv -m 'envios=850  semilla=424243'' had status 1”


 
-m 'envios=900  semilla=424243'

...

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c data-mining-gerencial-2025-a -f ./kaggle/KA6300_generencia_FE_gen_testB2_1200.csv -m 'envios=1200  semilla=424243'' had status 1”


 
-m 'envios=1250  semilla=424243'

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c data-mining-gerencial-2025-a -f ./kaggle/KA6300_generencia_FE_gen_testB2_1250.csv -m 'envios=1250  semilla=424243'' had status 1”


 
-m 'envios=1300  semilla=424243'

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c data-mining-gerencial-2025-a -f ./kaggle/KA6300_generencia_FE_gen_testB2_1300.csv -m 'envios=1300  semilla=424243'' had status 1”


In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Aug 13 06:14:52 AM 2025"